# Fletcher Project: Tweets Summarizer

In [7]:
import nltk
from nltk.tokenize import sent_tokenize
from nltk.tokenize import TreebankWordTokenizer
from nltk.tokenize import word_tokenize
from nltk.tokenize import wordpunct_tokenize
from nltk.tag import pos_tag
from nltk.chunk import ne_chunk
from nltk.corpus import treebank_chunk
from textblob import TextBlob
import pymongo
from nltk.corpus import stopwords
from nltk.corpus import movie_reviews
from nltk.util import ngrams
from nltk.tokenize import sent_tokenize, word_tokenize
from collections import defaultdict
from operator import itemgetter
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from __future__ import division, unicode_literals
import operator
import requests
from requests_oauthlib import OAuth1
from pymongo import MongoClient
import tweepy
import cnfg
import numpy as np
import re

## Opening Twitter API

In [8]:
config = cnfg.load(".twitter_config")

In [9]:
auth = tweepy.OAuthHandler(config['consumer_key'],
              config['consumer_secret'])
auth.set_access_token(config['access_token'],
              config['access_token_secret'])

In [10]:
api = tweepy.API(auth)

## Choosing Twitter Account

In [91]:
account = 'jk_rowling'

## Getting Tweeter Data into a List

In [92]:
data_tweets = api.user_timeline(account, count=200)
tweets = []
tweets_text = []
for tweet in data_tweets:
    tweets.append(tweet.text)
len(tweets)

196

In [93]:
tweets

[u'This is FANTASTIC, Johnnie, thank you! https://t.co/pTbDFyyUjf',
 u'@Johnnie_Rowling Good luck Johnnie! X',
 u"@JWaus I do indeed!  Take care of yourself, you're extremely important!  xxx",
 u'@JWaus Thank you!  Get rested and get well soon xxx',
 u'@HalesE xxx',
 u'RT @WWF: Newly discovered wasp named after the soul-sucking dementors in Harry Potter. Thanks for the inspiration @jk_rowling! http://t.co/\u2026',
 u"I don't care about WBC.  I think it's important that scared gay kids who aren't out yet see hate speech challenged. https://t.co/XumjDmEjuw",
 u'.@WBCsigns Alas, the sheer awesomeness of such a union in such a place would blow your tiny bigoted minds out of your thick sloping skulls.',
 u'RT @Okwonga: When #MikeBrown died in #Ferguson, the story was all over the news. The same should be true for #ShekuBayoh. http://t.co/z6Elh\u2026',
 u"RT @missingpeople: The morning after #TheBigTweet and we've got some refreshed info for you... you managed over 95,000 retweets in 24 hour

## Storing Tweets into Mongo DB

In [94]:
client = MongoClient()

In [95]:
datatweets = client.dsbc.datatweets

In [96]:
datatweets.remove( { } )

{u'n': 200, u'ok': 1}

In [97]:
datatweets = client.dsbc.datatweets
for k in tweets:
    datatweets_d = {'text': k}
    datatweets.insert(datatweets_d)

In [98]:
tweets = []
cursor = datatweets.find({})
for d in range(datatweets.count()):
    tweet_text = cursor.next()
    print tweet_text
    tweets.append(tweet_text['text'])
tweets

{u'text': u'.@simonmrzet X', u'_id': ObjectId('55691856bb9ab40227498835')}
{u'text': u'This is FANTASTIC, Johnnie, thank you! https://t.co/pTbDFyyUjf', u'_id': ObjectId('55691856bb9ab40227498786')}
{u'text': u'@Johnnie_Rowling Good luck Johnnie! X', u'_id': ObjectId('55691856bb9ab40227498787')}
{u'text': u"@JWaus I do indeed!  Take care of yourself, you're extremely important!  xxx", u'_id': ObjectId('55691856bb9ab40227498788')}
{u'text': u'@JWaus Thank you!  Get rested and get well soon xxx', u'_id': ObjectId('55691856bb9ab40227498789')}
{u'text': u'@HalesE xxx', u'_id': ObjectId('55691856bb9ab4022749878a')}
{u'text': u'@harryjjj @AngieCoiro @UberFacts http://t.co/vs5Vtgi1s7 :)', u'_id': ObjectId('55691856bb9ab40227498793')}
{u'text': u'@triwizarding Beyonc\xe9 is my Beyonc\xe9, but YOU are my JK Rowling.  Weird.', u'_id': ObjectId('55691856bb9ab402274987b7')}
{u'text': u'@jeromelingad Either way suits me!', u'_id': ObjectId('55691856bb9ab402274987bf')}
{u'text': u'@Steven_Strauss htt

[u'.@simonmrzet X',
 u'This is FANTASTIC, Johnnie, thank you! https://t.co/pTbDFyyUjf',
 u'@Johnnie_Rowling Good luck Johnnie! X',
 u"@JWaus I do indeed!  Take care of yourself, you're extremely important!  xxx",
 u'@JWaus Thank you!  Get rested and get well soon xxx',
 u'@HalesE xxx',
 u'@harryjjj @AngieCoiro @UberFacts http://t.co/vs5Vtgi1s7 :)',
 u'@triwizarding Beyonc\xe9 is my Beyonc\xe9, but YOU are my JK Rowling.  Weird.',
 u'@jeromelingad Either way suits me!',
 u'@Steven_Strauss http://t.co/v2oYDFUqEB ;)',
 u"Keep RTing, wonderful people, and if you're in the UK don't forget to keep an eye out for the missing. @missingpeople #TheBigTweet",
 u'RT @missingpeople: Help in the search &amp; retweet our appeal for Puong, 14, missing from #Warwickshire http://t.co/nXQSolrHdr #TheBigTweet ht\u2026',
 u"RT @missingpeople: Kaylie's missing from #Sidcup #Kent. RT and help find her http://t.co/wLMc1rieWO #TheBigTweet #ThanksICAP http://t.co/HM\u2026",
 u'RT @missingpeople: Please help sea

## Summarizing all most recent 200 Tweets

In [99]:
stopw = [str('http'), str('https'), str('['), str(']'), str('rt'), str('%'), str('@'), str('#')]
stopngram = stopwords.words('english')
stopngram += ['.', ',', '(', ')', "'", '"', 'http', 'RT']

In [100]:
adjectives = {}
nouns = {}
adverbs = {}
verbs = {}
for t in tweets:
    words = word_tokenize(t)
    for p in pos_tag(words):
        tags = p[1][:2]
        keys = p[0].encode('utf8')
        keys = keys.lower()   
        if keys not in stopw and str('/') not in keys and "\\" not in r"%r" % keys:
            if tags == 'NN':
                nouns.setdefault(keys,0)
                nouns[keys] += 1

## Most 10 used Nouns, Adverbs in all 200 recent Tweets

In [101]:
for words, count in sorted(nouns.items(), key = itemgetter(1), reverse=True)[:10]:
    print '%20s %i' % (words, count)

         thebigtweet 56
       missingpeople 52
          thanksicap 30
          jk_rowling 29
              please 15
                help 13
                   x 11
             missing 11
              people 10
             croydon 8


## Most 10 used 2-grams, 3-grams, 4-grams in all 200 recent Tweets

In [102]:
counter = defaultdict(int)
n = 2
for doc in tweets:
    words = TextBlob(doc).words
    words = [w for w in words if w not in stopngram]
    bigrams = ngrams(words, n)
    for gram in bigrams:
        counter[gram] += 1
            
for gram, count in sorted(counter.items(), key = itemgetter(1), reverse=True)[:10]:
    phrase = " ".join(gram)
    print '%20s %i' % (phrase, count)

TheBigTweet ThanksICAP 29
          's missing 15
        went missing 10
           help find 10
     missing Croydon 8
                I 'm 8
missingpeople Please 6
         Please help 6
               I 've 6
             116 000 6


In [103]:
counter = defaultdict(int)
n = 3
for doc in tweets:
    words = TextBlob(doc).words
    words = [w for w in words if w not in stopngram]
    bigrams = ngrams(words, n)
    for gram in bigrams:
        counter[gram] += 1
            
for gram, count in sorted(counter.items(), key = itemgetter(1), reverse=True)[:10]:
    phrase = " ".join(gram)
    print '%20s %i' % (phrase, count)

missingpeople Please help 5
  's missing Croydon 4
        116 000 info 4
        help us find 4
        Call 116 000 4
TheBigTweet ThanksICAP t… 3
    Please help find 2
       amp help find 2
       amp need help 2
    could poster All 2


In [104]:
counter = defaultdict(int)
n = 4
for doc in tweets:
    words = TextBlob(doc).words
    words = [w for w in words if w not in stopngram]
    bigrams = ngrams(words, n)
    for gram in bigrams:
        counter[gram] += 1
            
for gram, count in sorted(counter.items(), key = itemgetter(1), reverse=True)[:10]:
    phrase = " ".join(gram)
    print '%20s %i' % (phrase, count)

   Call 116 000 info 3
missingpeople Kevin 's missing 2
want see return safely 2
RTing missingpeople 's appeals 2
please could poster All 2
jk_rowling You always welcome 2
missingpeople Please help find 2
missing Manchester since 2012 2
Today International Missing Children 2
jk_rowling please could poster 2


## Selecting a Keyword

In [127]:
keyword = 'potter'

In [128]:
keyword = str(keyword)
keyword = keyword.lower()
keyword

'potter'

In [129]:
tweets_match = []
for r in tweets:
    if keyword in r.lower():
        tweets_match.append(r)

len(tweets_match)

4

In [130]:
adjectives = {}
nouns = {}
adverbs = {}
verbs = {}
for t in tweets_match:
    words = word_tokenize(t)
    for p in pos_tag(words):
        tags = p[1][:2]
        keys = p[0].encode('utf8')
        keys = keys.lower()   
        if keys not in stopw and str('/') not in keys and keyword not in keys and "\\" not in r"%r" % keys:
            if tags == 'JJ':
                adjectives.setdefault(keys,0)
                adjectives[keys] += 1
            if tags == 'NN':
                nouns.setdefault(keys,0)
                nouns[keys] += 1
            if tags == 'RB':
                adverbs.setdefault(keys,0)
                adverbs[keys] += 1
            if tags == 'VB':
                verbs.setdefault(keys,0)
                verbs[keys] += 1

## Most 5 used Verbs, Adjectives, Nouns, Adverbs in Tweets with Keyword

In [131]:
for words, count in sorted(verbs.items(), key = itemgetter(1), reverse=True)[:5]:
    print '%20s %i' % (words, count)

          discovered 1
               named 1
            prepared 1
           celebrate 1
              superb 1


In [132]:
for words, count in sorted(adjectives.items(), key = itemgetter(1), reverse=True)[:5]:
    print '%20s %i' % (words, count)

             current 1
                cast 1
        soul-sucking 1
              latest 1


In [133]:
for words, count in sorted(nouns.items(), key = itemgetter(1), reverse=True)[:5]:
    print '%20s %i' % (words, count)

            housecup 2
           standings 1
               point 1
               house 1
                 wwf 1


In [134]:
for words, count in sorted(adverbs.items(), key = itemgetter(1), reverse=True)[:5]:
    print '%20s %i' % (words, count)

                 not 1
                 yet 1


## Most 5 used 2-grams, 3-grams, 4-grams in Tweets with Keyword

In [135]:
counter = defaultdict(int)
n = 2
for doc in tweets_match:
    words = TextBlob(doc).words
    words = [w for w in words if w not in stopngram]
    bigrams = ngrams(words, n)
    for gram in bigrams:
        counter[gram] += 1
            
for gram, count in sorted(counter.items(), key = itemgetter(1), reverse=True)[:5]:
    phrase = " ".join(gram)
    print '%20s %i' % (phrase, count)

         skill we’ve 1
       pottermore To 1
Congratulations Slytherin 1
 pottermore HouseCup 1
  named soul-sucking 1


In [136]:
counter = defaultdict(int)
n = 3
for doc in tweets_match:
    words = TextBlob(doc).words
    words = [w for w in words if w not in stopngram]
    bigrams = ngrams(words, n)
    for gram in bigrams:
        counter[gram] += 1
            
for gram, count in sorted(counter.items(), key = itemgetter(1), reverse=True)[:5]:
    phrase = " ".join(gram)
    print '%20s %i' % (phrase, count)

discovered wasp named 1
Slytherins Through Ages’ 1
prepared look ‘Famous 1
point standings pottermore 1
   remember it’s yet 1


In [137]:
counter = defaultdict(int)
n = 4
for doc in tweets_match:
    words = TextBlob(doc).words
    words = [w for w in words if w not in stopngram]
    bigrams = ngrams(words, n)
    for gram in bigrams:
        counter[gram] += 1
            
for gram, count in sorted(counter.items(), key = itemgetter(1), reverse=True)[:5]:
    phrase = " ".join(gram)
    print '%20s %i' % (phrase, count)

we’ve prepared look ‘Famous 1
SlytherinPride superb skill we’ve 1
pottermore HouseCup – remember 1
Harry Potter Thanks inspiration 1
pottermore To celebrate SlytherinPride 1
